In [22]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator as data_augment

In [23]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade_left = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lefteye_2splits.xml')



In [24]:
import cv2
import numpy as np

def detect_face_and_eyes(frame, target_size=(224, 224)):
    normalized_frame = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX)
    frame_u8 = np.uint8(normalized_frame)
    
    gray = cv2.cvtColor(frame_u8, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes_left = eye_cascade_left.detectMultiScale(roi_gray)

        if len(eyes_left) > 0:
            left_eye = roi_gray[eyes_left[0][1]:eyes_left[0][1]+eyes_left[0][3], eyes_left[0][0]:eyes_left[0][0]+eyes_left[0][2]]
            face = roi_gray

            # Resize to target size
            left_eye = cv2.resize(left_eye, target_size)
            face = cv2.resize(face, target_size)

            # Expand dimensions to fit target shape (224, 224, 1)
            left_eye = np.expand_dims(left_eye, axis=2)
            face = np.expand_dims(face, axis=2)

            return face, left_eye

    return None, None


In [25]:
# Function to process frames and labels with face detection
def process_frames_with_eyes(frames_and_labels):
    # Separate features and labels

    frames, labels = zip(*frames_and_labels)
    left_eye_frames = []
    face_frames = []
    labels = []
    
    for i in range(len(frames)):
    
        # Detect face and eyes
        frame = frames[i]
        face_regions, left_eye_regions = detect_face_and_eyes(frame)

        for face, left_eye in zip(face_regions, left_eye_regions):
            # Resize left eye region
            
            if (left_eye is not None and face is not None):
                # Normalize pixel values
                left_eye = left_eye / 255.0
                face = face / 255.0

                left_eye_frames.append(left_eye)
                face_frames.append(face)
                labels.append(labels[i])

    return face_frames, left_eye_frames, labels


In [26]:
def process_images_from_folder(folder_path, label):

    image_data = []
    for filename in os.listdir(folder_path):

        if filename.endswith(".jpg") or filename.endswith(".png"): 
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)  # Read in BGR format
            image_data.extend(zip(img, label))
            
    return image_data

In [27]:
# Example usage
dataset_folder = "Training Data"

# Load and process drowsy videos
drowsy_data = process_images_from_folder(os.path.join(dataset_folder, "Drowsy"), "drowsy")


In [28]:
# Load and process alert videos
alert_data = process_images_from_folder(os.path.join(dataset_folder, "Non Drowsy"), "alert")

In [29]:
# Combine alert and drowsy data
all_data = alert_data + drowsy_data
np.random.shuffle(all_data)


In [30]:
# Process frames with face detection
face_frames, left_eye_frames, labels = process_frames_with_eyes(all_data)

error: OpenCV(4.9.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x59191d0d::Set<1,-1,-1>,struct cv::impl::A0x59191d0d::Set<0,2,5>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [ ]:
# Convert to numpy arrays
face_frames = np.array(face_frames)
left_eye_frames = np.array(left_eye_frames)

# Convert labels to numeric format
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_encoded = np.array(labels_encoded)

In [ ]:
np.savenpz('eye_data.npz', left_eye_frames = left_eye_frames, face_frames  = face_frames, labels = labels_encoded)

In [ ]:
#data augmetation 
data_generate_training = data_augment (rescale=1./255, 
                              shear_range = 0.2,
                              zoom_range = 0.2,
                              fill_mode = "nearest",
                              horizontal_flip = True,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              validation_split = 0.15)

data_generate_test = data_augment(rescale = 1./255)


In [ ]:
#data preprocessing and augmentation
traind = data_generate_training.flow_from_directory("Training Data",
                                          target_size = (224, 224),
                                          seed = 123,
                                          batch_size = 32,
                                          subset = "training")

testd = data_generate_training.flow_from_directory("Training Data"/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset4(DDD4",
                                          target_size = (227, 227),
                                          seed = 123,
                                          batch_size = 32,
                                          subset = "validation")